In [3]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import re
import os
from math import isnan
from datetime import datetime

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        return conn



con = create_connection(r"gilsontsl.db")

2.6.0


In [2]:
con.close()

In [10]:
TABLE_NAME = 'GILSON_SET_TABLE'

df = pd.read_sql_query(f"SELECT * from {TABLE_NAME}", con)

# Verify that result of SQL query is stored in the dataframe
print(df.head())

                     RUN_DATE          ID  SAMPLE_NUMBER SAMPLE_NAME  \
0  2020-04-06 16:47:32.317022  1520030001              0    M21535_5   
1  2020-04-06 16:47:32.343616  1520030001              1    M21551_5   
2  2020-04-06 16:47:32.344137  1520030001              2    M21523_5   
3  2020-04-06 16:47:32.345212  1520030001              3    M24173_2   
4  2020-04-06 16:47:32.346140  1520030001              4    M23131_6   

  TUBE_BARCODE BROOKS_BARCODE  
0   FB02646205            127  
1   FB02646225            127  
2   FB02646185            127  
3   FB03658814            127  
4   FB02699823            127  


In [12]:
pd.read_sql_query('SELECT DISTINCT ID FROM GILSON_SET_TABLE', con)

,ID
0,1520030001
1,1520030002
2,1520030003


In [5]:
d_test = pd.read_csv('test_files/1520030001_secStg_127.tsl',sep='\t')
d_test.head()

,"SkipPause[True,18,0]","MethodName[True,239,1]","SampleName[True,108,2]","SampleAmount[False,78,3]","SampleDescription[False,120,4]","SampleId[False,90,5]","InjectionYesNo[False,72,6]","PeakInformation[False,104,7]","SampleInjection[False,138,8]","NOTES_STRING[False,68,9]","#Sample Well[True,109,10]","#Plate_Sample[True,115,13]"
0,RUN,4 ML STARTUP,Sample,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
1,RUN,4ML_10mm_2ND STAGE_FLUSH,Flush,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
2,RUN,4ML_10mm_2ND STAGE_STD,6-Cmpd Std,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
3,RUN,4ML_10mm_2ND STAGE,M21535_5,0,FB02646205,NaN,YES,UNKNOWN,...,127.0,1,P01S01
4,RUN,4ML_10mm_2ND STAGE,M21551_5,0,FB02646225,NaN,YES,UNKNOWN,...,127.0,2,P01S02


In [6]:
col_names = [re.search(r'^.+?(?=\[)', c).group(0) for c in d_test.columns]
col_names

['SkipPause',
 'MethodName',
 'SampleName',
 'SampleAmount',
 'SampleDescription',
 'SampleId',
 'InjectionYesNo',
 'PeakInformation',
 'SampleInjection',
 'NOTES_STRING',
 '#Sample Well',
 '#Plate_Sample']

In [9]:
sqlite_create_table_query = '''CREATE TABLE GILSON_TSL_TABLE (NUMBER INTEGER NOT NULL,RUNDATE TIMESTAMP,PLATEID TEXT NOT NULL,'''

In [10]:
sqlite_create_table_query

'CREATE TABLE GILSON_TSL_TABLE (NUMBER INTEGER NOT NULL,RUNDATE TIMESTAMP,PLATEID TEXT NOT NULL,'

In [11]:
comma = ','
for i,c in enumerate(col_names):
    if i == len(col_names) - 1:
        comma = ''
    if ord(c[0]) < 65:
        c = c[1:]
    sqlite_create_table_query += f'{c.upper()} TEXT NOT NULL{comma}'
    
    
sqlite_create_table_query += ');'

In [12]:
sqlite_create_table_query

'CREATE TABLE GILSON_TSL_TABLE (NUMBER INTEGER NOT NULL,RUNDATE TIMESTAMP,PLATEID TEXT NOT NULL,SKIPPAUSE TEXT NOT NULL,METHODNAME TEXT NOT NULL,SAMPLENAME TEXT NOT NULL,SAMPLEAMOUNT TEXT NOT NULL,SAMPLEDESCRIPTION TEXT NOT NULL,SAMPLEID TEXT NOT NULL,INJECTIONYESNO TEXT NOT NULL,PEAKINFORMATION TEXT NOT NULL,SAMPLEINJECTION TEXT NOT NULL,NOTES_STRING TEXT NOT NULL,SAMPLE WELL TEXT NOT NULL,PLATE_SAMPLE TEXT NOT NULL);'

In [15]:
con.execute(sqlite_create_table_query)

In [13]:
sqlite_delete_table_query = '''DROP TABLE GILSON_TSL_TABLE;'''

In [14]:
con.execute(sqlite_delete_table_query)

In [16]:
pd.read_sql_query('SELECT * FROM GILSON_TSL_TABLE', con)

,NUMBER,RUNDATE,PLATEID,SKIPPAUSE,METHODNAME,SAMPLENAME,SAMPLEAMOUNT,SAMPLEDESCRIPTION,SAMPLEID,INJECTIONYESNO,PEAKINFORMATION,SAMPLEINJECTION,NOTES_STRING,SAMPLE,PLATE_SAMPLE


In [47]:
dir_fi = [fi for fi in os.listdir('test_files/')]
dir_fi

['1520060002_secStg_15200600020.tsl',
 '1520030001_secStg_127.tsl',
 '1520030002_secStg_134.tsl',
 'gilson_set_file_127.txt',
 'gilson_set_file_134.txt',
 '1520030003_secStg_138.tsl',
 'gilson_set_file_138.txt',
 'Excel_gilson_script_test.xlsx',
 '1520060001_secStg_15200600010.tsl']

In [104]:
dtest = pd.read_csv('/Volumes/npsg/tecan/SourceData/SecondStage/15200300_01_02_127_134.tsl',sep='\t')
dtest

,"SkipPause[True,18,0]","MethodName[True,239,1]","SampleName[True,108,2]","SampleAmount[False,78,3]","SampleDescription[False,120,4]","SampleId[False,90,5]","InjectionYesNo[False,72,6]","PeakInformation[False,104,7]","SampleInjection[False,138,8]","NOTES_STRING[False,68,9]","#Sample Well[True,109,10]","#Plate_Sample[True,115,13]"
0,RUN,4 ML STARTUP,Sample,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
1,RUN,4ML_10mm_2ND STAGE_FLUSH,Flush,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
2,RUN,4ML_10mm_2ND STAGE_STD,6-Cmpd Std,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
3,RUN,4ML_10mm_2ND STAGE,M21535_5,0,FB02646205,NaN,YES,UNKNOWN,...,127.0,1,P01S01
4,RUN,4ML_10mm_2ND STAGE,M21551_5,0,FB02646225,NaN,YES,UNKNOWN,...,127.0,2,P01S02
5,RUN,4ML_10mm_2ND STAGE,M21523_5,0,FB02646185,NaN,YES,UNKNOWN,...,127.0,3,P01S03
6,RUN,4ML_10mm_2ND STAGE,M24173_2,0,FB03658814,NaN,YES,UNKNOWN,...,127.0,4,P01S04
7,RUN,4ML_10mm_2ND STAGE,M23131_6,0,FB02699823,NaN,YES,UNKNOWN,...,127.0,5,P02S01
8,RUN,4ML_10mm_2ND STAGE,M23127_6,0,FB02699828,NaN,YES,UNKNOWN,...,127.0,6,P02S02
9,RUN,4ML_10mm_2ND STAGE,L7547_3,0,FB02928213,NaN,YES,UNKNOWN,...,127.0,7,P02S03


In [103]:
dtest = pd.read_csv('/Volumes/npsg/tecan/SourceData/SecondStage/15200600_01_02_15200600010_15200600020.tsl',sep='\t')
dtest

,"SkipPause[True,18,0]","MethodName[True,239,1]","SampleName[True,108,2]","SampleAmount[False,78,3]","SampleDescription[False,120,4]","SampleId[False,90,5]","InjectionYesNo[False,72,6]","PeakInformation[False,104,7]","SampleInjection[False,138,8]","NOTES_STRING[False,68,9]","#Sample Well[True,109,10]","#Plate_Sample[True,115,13]"
0,RUN,4 ML STARTUP,Sample,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
1,RUN,4ML_10mm_2ND STAGE_FLUSH,Flush,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
2,RUN,4ML_10mm_2ND STAGE_STD,6-Cmpd Std,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
3,RUN,4ML_10mm_2ND STAGE,L129525_6,0,FB04587339,NaN,YES,UNKNOWN,...,1.520060e+10,1,P01S01
4,RUN,4ML_10mm_2ND STAGE,L48809_5,0,FB04251130,NaN,YES,UNKNOWN,...,1.520060e+10,2,P01S02
5,RUN,4ML_10mm_2ND STAGE,L129575_6,0,FB04716251,NaN,YES,UNKNOWN,...,1.520060e+10,3,P01S03
6,RUN,4ML_10mm_2ND STAGE,L48781_7,0,FB04250855,NaN,YES,UNKNOWN,...,1.520060e+10,4,P01S04
7,RUN,4ML_10mm_2ND STAGE,L99277_7,0,FB04242223,NaN,YES,UNKNOWN,...,1.520060e+10,5,P02S01
8,RUN,4ML_10mm_2ND STAGE,L77355_6,0,FB04363141,NaN,YES,UNKNOWN,...,1.520060e+10,6,P02S02
9,RUN,4ML_10mm_2ND STAGE,L22855_6,0,FB04575204,NaN,YES,UNKNOWN,...,1.520060e+10,7,P02S03


In [43]:
d2_test = pd.read_csv('test_files/1520060001_secStg_15200600010.tsl',sep='\t')
d2_test.head()

,"SkipPause[True,18,0]","MethodName[True,239,1]","SampleName[True,108,2]","SampleAmount[False,78,3]","SampleDescription[False,120,4]","SampleId[False,90,5]","InjectionYesNo[False,72,6]","PeakInformation[False,104,7]","SampleInjection[False,138,8]","NOTES_STRING[False,68,9]","#Sample Well[True,109,10]","#Plate_Sample[True,115,13]"
0,RUN,4 ML STARTUP,Sample,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
1,RUN,4ML_10mm_2ND STAGE_FLUSH,Flush,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
2,RUN,4ML_10mm_2ND STAGE_STD,6-Cmpd Std,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
3,RUN,4ML_10mm_2ND STAGE,L129525_6,0,FB04587339,NaN,YES,UNKNOWN,...,1.520060e+10,1,P01S01
4,RUN,4ML_10mm_2ND STAGE,L48809_5,0,FB04251130,NaN,YES,UNKNOWN,...,1.520060e+10,2,P01S02


In [81]:
d_test

,"SkipPause[True,18,0]","MethodName[True,239,1]","SampleName[True,108,2]","SampleAmount[False,78,3]","SampleDescription[False,120,4]","SampleId[False,90,5]","InjectionYesNo[False,72,6]","PeakInformation[False,104,7]","SampleInjection[False,138,8]","NOTES_STRING[False,68,9]","#Sample Well[True,109,10]","#Plate_Sample[True,115,13]"
0,RUN,4 ML STARTUP,Sample,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
1,RUN,4ML_10mm_2ND STAGE_FLUSH,Flush,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
2,RUN,4ML_10mm_2ND STAGE_STD,6-Cmpd Std,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
3,RUN,4ML_10mm_2ND STAGE,M21535_5,0,FB02646205,NaN,YES,UNKNOWN,...,127.0,1,P01S01
4,RUN,4ML_10mm_2ND STAGE,M21551_5,0,FB02646225,NaN,YES,UNKNOWN,...,127.0,2,P01S02
5,RUN,4ML_10mm_2ND STAGE,M21523_5,0,FB02646185,NaN,YES,UNKNOWN,...,127.0,3,P01S03
6,RUN,4ML_10mm_2ND STAGE,M24173_2,0,FB03658814,NaN,YES,UNKNOWN,...,127.0,4,P01S04
7,RUN,4ML_10mm_2ND STAGE,M23131_6,0,FB02699823,NaN,YES,UNKNOWN,...,127.0,5,P02S01
8,RUN,4ML_10mm_2ND STAGE,M23127_6,0,FB02699828,NaN,YES,UNKNOWN,...,127.0,6,P02S02
9,RUN,4ML_10mm_2ND STAGE,L7547_3,0,FB02928213,NaN,YES,UNKNOWN,...,127.0,7,P02S03


In [97]:
with open('test_files/1520030001_secStg_127.tsl','r') as f:
    t = f.readlines()

In [98]:
len(t)

38

In [99]:
t

['SkipPause[True,18,0]\tMethodName[True,239,1]\tSampleName[True,108,2]\tSampleAmount[False,78,3]\tSampleDescription[False,120,4]\tSampleId[False,90,5]\tInjectionYesNo[False,72,6]\tPeakInformation[False,104,7]\tSampleInjection[False,138,8]\tNOTES_STRING[False,68,9]\t#Sample Well[True,109,10]\t#Plate_Sample[True,115,13]\n',
 '\n',
 'RUN\t4 ML STARTUP\tSample\t0\t\t\tYES\tUNKNOWN\t...\t\t0\t\n',
 'RUN\t4ML_10mm_2ND STAGE_FLUSH\tFlush\t0\t\t\tYES\tUNKNOWN\t...\t\t0\t\n',
 'RUN\t4ML_10mm_2ND STAGE_STD\t6-Cmpd Std\t0\t\t\tYES\tUNKNOWN\t...\t\t0\t\n',
 'RUN\t4ML_10mm_2ND STAGE\tM21535_5\t0\tFB02646205\t\tYES\tUNKNOWN\t...\t127\t1\tP01S01\n',
 'RUN\t4ML_10mm_2ND STAGE\tM21551_5\t0\tFB02646225\t\tYES\tUNKNOWN\t...\t127\t2\tP01S02\n',
 'RUN\t4ML_10mm_2ND STAGE\tM21523_5\t0\tFB02646185\t\tYES\tUNKNOWN\t...\t127\t3\tP01S03\n',
 'RUN\t4ML_10mm_2ND STAGE\tM24173_2\t0\tFB03658814\t\tYES\tUNKNOWN\t...\t127\t4\tP01S04\n',
 'RUN\t4ML_10mm_2ND STAGE\tM23131_6\t0\tFB02699823\t\tYES\tUNKNOWN\t...\t127\t5\t

37 - RUN	4 ML SHUTDOWN	Sample	0	SHUTDOWN		YES	UNKNOWN	...		0

36 - RUN	4ML_10mm_2ND STAGE_STD	6-Cmpd Std	0	6-Cmpd Std		YES	UNKNOWN	...		0

35 - RUN	4ML_10mm_2ND STAGE_FLUSH	Sample	0	Flush		YES	UNKNOWN	...		0

34 - RUN	4ML_10mm_2ND STAGE_STD	6-Cmpd Std	0			YES	UNKNOWN	...		0	

33 - RUN	4ML_10mm_2ND STAGE_FLUSH	Flush	0			YES	UNKNOWN	...		0	

32 - RUN	4ML_10mm_2ND STAGE	L5885_5	0	FB02931250		YES	UNKNOWN	...	127	24	P06S04
 *** 
 *** 	24	


In [58]:
def grab_rows(fi_content_df):
    row_idx=[]
    for i in range(fi_content_df.shape[0]):
        try:
            if isnan(fi_content_df['#Plate_Sample[True,115,13]'][i]):
                pass#print('nothing')
        except TypeError:
             row_idx.append(i)
            #print('good')
    return row_idx

In [74]:
def grab_idx_sample(fi_content_df):
    return [i for i,s in enumerate(fi_content_df.columns) if s == '#Sample Well[True,109,10]'][0]

In [79]:
def imbue_rows(dir_fi):
    nbr = 0 
    split_fi_name = dir_fi.split('_')
    plateid = split_fi_name[0]
    suffix = split_fi_name[2].split('.')[0]
    fi_content_df = pd.read_csv(dir_fi, sep='\t')
    num_rows = len(fi_content_df.index)
    row_idx = grab_rows(fi_content_df)
    len_rows = len(row_idx)
    len_cols = len(fi_content_df.columns)
    if len_rows % 4 != 0:
        len_rows = len_rows + (4-len_rows%4)
    blank_sample_idx = grab_idx_sample(fi_content_df)
    
    for i in range(len_rows): 
        nbr += 1
        row_data = ["BLANK"]*len_cols
        row_data[blank_sample_idx] = nbr
        row_data[0] = nbr
        try:
            row_data =  fi_content_df.iloc[row_idx[i]].values.tolist()
            first_2_vals = [nbr, datetime.now().strftime("%Y-%b-%d %H:%M:%S"), split_fi_name[0]]
            row_data = first_2_vals + row_data
        except IndexError as e:
            pass
        print(row_data)
        #c.executemany("INSERT INTO GILSON_SET_TABLE VALUES (?,?,?,?,?,?)", row_data)


In [80]:
imbue_rows(f'test_files/{dir_fi[-1]}')

[1, '2020-Jun-25 17:57:43', 'test', 'RUN', '4ML_10mm_2ND STAGE', 'L129525_6', 0, 'FB04587339', nan, 'YES', 'UNKNOWN', '...', 15200600010.0, 1, 'P01S01']
[2, '2020-Jun-25 17:57:43', 'test', 'RUN', '4ML_10mm_2ND STAGE', 'L48809_5', 0, 'FB04251130', nan, 'YES', 'UNKNOWN', '...', 15200600010.0, 2, 'P01S02']
[3, '2020-Jun-25 17:57:43', 'test', 'RUN', '4ML_10mm_2ND STAGE', 'L129575_6', 0, 'FB04716251', nan, 'YES', 'UNKNOWN', '...', 15200600010.0, 3, 'P01S03']
[4, '2020-Jun-25 17:57:43', 'test', 'RUN', '4ML_10mm_2ND STAGE', 'L48781_7', 0, 'FB04250855', nan, 'YES', 'UNKNOWN', '...', 15200600010.0, 4, 'P01S04']
[5, '2020-Jun-25 17:57:43', 'test', 'RUN', '4ML_10mm_2ND STAGE', 'L99277_7', 0, 'FB04242223', nan, 'YES', 'UNKNOWN', '...', 15200600010.0, 5, 'P02S01']
[6, '2020-Jun-25 17:57:43', 'test', 'RUN', '4ML_10mm_2ND STAGE', 'L77355_6', 0, 'FB04363141', nan, 'YES', 'UNKNOWN', '...', 15200600010.0, 6, 'P02S02']
[7, '2020-Jun-25 17:57:43', 'test', 'RUN', '4ML_10mm_2ND STAGE', 'L22855_6', 0, 'FB045